In [ ]:
# === F3Net (balanced_frames_FF++) — dual input modes (FFT & RGB) + small robust selection ===
# Prints ONLY: AUC | EER | AP  (+ one info line)

from google.colab import drive
drive.mount('/content/drive', force_remount=False)

# ---------- paths ----------
import os, re, sys, subprocess, numpy as np, pandas as pd, cv2
from PIL import Image

ROOT = "/content/drive/MyDrive" if os.path.isdir("/content/drive/MyDrive") else "/content/drive/My Drive"
DATASET_ROOT = f"{ROOT}/balanced_frames_FF++"
REAL_DIR = f"{DATASET_ROOT}/real"
FAKE_DIR = f"{DATASET_ROOT}/fake"
F3NET_WEIGHTS = f"{ROOT}/DeepfakeBench_weights/f3net_best.pth"
DATASET_NAME = "balanced_frames_FF++"

assert os.path.isdir(REAL_DIR), f"Missing: {REAL_DIR}"
assert os.path.isdir(FAKE_DIR), f"Missing: {FAKE_DIR}"
assert os.path.isfile(F3NET_WEIGHTS), f"Missing: {F3NET_WEIGHTS}"

# ---------- deps ----------
def _pipq(*pkgs):
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", *pkgs], check=True)
try:
    import timm
except Exception:
    _pipq("timm==1.0.9"); import timm

import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve

# ---------- hardware + knobs ----------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = (device.type=="cuda")
softmax = torch.nn.Softmax(dim=1)

HEAVY_TTA   = True
CROP_SIZES  = [256, 280]    # add 240 if VRAM allows
MAX_CAP     = 160           # score up to 160 frames/video; we subset in selection
BATCH       = 24 if device.type=="cuda" else 8
NUM_WORKERS = 2 if device.type=="cuda" else 0

# ---------- list frames ----------
IMG_EXTS = (".jpg",".jpeg",".png",".bmp",".webp")
def list_imgs(d):
    return sorted([os.path.join(d,f) for f in os.listdir(d) if f.lower().endswith(IMG_EXTS)]) if os.path.isdir(d) else []
reals = list_imgs(REAL_DIR); fakes = list_imgs(FAKE_DIR)
assert len(reals) and len(fakes), f"No images. REAL={len(reals)} FAKE={len(fakes)}."

def infer_video_name(path):
    stem = os.path.splitext(os.path.basename(path))[0]
    m = re.split(r"_frame(\d+)$", stem)
    return m[0] if len(m)>1 and m[0] else re.sub(r"[_\\-]\\d+$","",stem)

def frame_index(path):
    m = re.search(r"_frame(\d+)", os.path.basename(path))
    return int(m.group(1)) if m else 10**9

def build_df(paths, label):
    rows=[{"path":p,"video_name":infer_video_name(p),"idx":frame_index(p),"label":label} for p in paths]
    df = pd.DataFrame(rows).sort_values(["video_name","idx"])
    df["video_name"]=df["video_name"].astype(object)
    df["idx"]=pd.to_numeric(df["idx"], errors="coerce").astype(int)
    df["label"]=pd.to_numeric(df["label"], errors="coerce").astype(int)
    return df

df_r = build_df(reals, 0)
df_f = build_df(fakes, 1)
df_all = pd.concat([df_r, df_f], ignore_index=True)
df_sel = (df_all.sort_values(["video_name","idx"]).groupby("video_name", as_index=False).head(MAX_CAP))

# ---------- preprocess (FFT & RGB) ----------
IMG_SIZE = 299
IMN_MEAN = np.array([0.485, 0.456, 0.406], dtype=np.float32)
IMN_STD  = np.array([0.229, 0.224, 0.225], dtype=np.float32)

def fft_logmag(gray_f32):
    F = np.fft.fft2(gray_f32); Fshift = np.fft.fftshift(F)
    mag = np.log1p(np.abs(Fshift))
    mag = mag / (mag.max() + 1e-8)
    return mag.astype(np.float32)

def prep_fft_rgb(path, out_size=IMG_SIZE):
    im = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if im is None: im = np.array(Image.open(path).convert("L"))
    im = cv2.resize(im, (out_size, out_size), interpolation=cv2.INTER_CUBIC).astype(np.float32)/255.0
    mag = fft_logmag(im)
    x = np.stack([mag,mag,mag], axis=2).transpose(2,0,1)  # 3ch
    x = (x - IMN_MEAN[:,None,None]) / IMN_STD[:,None,None]
    return torch.from_numpy(x.astype(np.float32))

def prep_rgb(path, out_size=IMG_SIZE):
    im = cv2.imread(path, cv2.IMREAD_COLOR)
    if im is None: im = cv2.cvtColor(np.array(Image.open(path).convert("RGB")), cv2.COLOR_RGB2BGR)
    im = cv2.resize(im, (out_size, out_size), interpolation=cv2.INTER_CUBIC).astype(np.float32)/255.0
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)  # to RGB
    x = im.transpose(2,0,1)  # CHW
    x = (x - IMN_MEAN[:,None,None]) / IMN_STD[:,None,None]
    return torch.from_numpy(x.astype(np.float32))

class FreqDatasetFFT(Dataset):
    def __init__(self, df): self.df = df.reset_index(drop=True)
    def __len__(self): return len(self.df)
    def __getitem__(self, i):
        r = self.df.iloc[i]
        return prep_fft_rgb(r["path"]), int(r["label"]), str(r["video_name"]), int(r["idx"])

class FreqDatasetRGB(Dataset):
    def __init__(self, df): self.df = df.reset_index(drop=True)
    def __len__(self): return len(self.df)
    def __getitem__(self, i):
        r = self.df.iloc[i]
        return prep_rgb(r["path"]), int(r["label"]), str(r["video_name"]), int(r["idx"])

# ---------- model + partial-load F3Net weights (Xception head) ----------
model = timm.create_model('legacy_xception', pretrained=True, num_classes=2)

def try_load_weights(model, ckpt_path, min_cover=0.5):
    ok=False; cover=0.0
    try:
        sd = torch.load(ckpt_path, map_location="cpu")
        if isinstance(sd, dict):
            for k in ("state_dict","model","net","weights","model_state","ema_state_dict"):
                if k in sd and isinstance(sd[k], dict):
                    sd = sd[k]; break
        clean={}
        if isinstance(sd, dict):
            for k,v in sd.items():
                if not isinstance(k,str): continue
                k2=k
                for pref in ("module.","model.","net.","backbone."):
                    if k2.startswith(pref): k2=k2[len(pref):]
                clean[k2]=v
            ms = model.state_dict()
            matched = {k:v for k,v in clean.items() if k in ms and ms[k].shape==v.shape}
            cover = len(matched)/max(1,len(ms))
            if cover >= min_cover:
                ms.update(matched); model.load_state_dict(ms, strict=False); ok=True
    except Exception as e:
        print("[warn] weight load:", e)
    return ok, cover

weights_loaded, coverage = try_load_weights(model, F3NET_WEIGHTS, min_cover=0.5)
model = model.to(device).eval()

# ---------- heavy TTA ----------
def ten_crops(x, crop):
    B,C,H,W = x.shape; ch=crop; cw=crop
    tl = x[...,0:ch,0:cw]; tr = x[...,0:ch,W-cw:W]
    bl = x[...,H-ch:H,0:cw]; br = x[...,H-ch:H,W-cw:W]
    cs = x[..., (H-ch)//2:(H+ch)//2, (W-cw)//2:(W+cw)//2]
    flips = [torch.flip(t, dims=[3]) for t in (tl,tr,bl,br,cs)]
    return [tl,tr,bl,br,cs] + flips

@torch.no_grad()
def forward_tta(xb):
    use_amp = (device.type=="cuda")
    logits_sum=None
    for crop in CROP_SIZES:
        for p in ten_crops(xb, crop):
            with torch.amp.autocast('cuda', enabled=use_amp):
                out = model(p)
            logits_sum = out if logits_sum is None else (logits_sum + out)
    return logits_sum / float(len(CROP_SIZES)*10)

# ---------- score one mode ----------
@torch.no_grad()
def score_mode(df, mode="fft"):
    if mode=="fft":
        ds = FreqDatasetFFT(df)
    elif mode=="rgb":
        ds = FreqDatasetRGB(df)
    else:
        raise ValueError("mode must be 'fft' or 'rgb'")
    loader = DataLoader(ds, batch_size=BATCH, shuffle=False,
                        num_workers=NUM_WORKERS, pin_memory=(device.type=="cuda"))
    vnames, idxs, probs, logits1m0, labels = [], [], [], [], []
    for xb, yb, vb, ib in loader:
        xb = xb.to(device, non_blocking=(device.type=="cuda"))
        logits = forward_tta(xb)                # [B,2]
        p = softmax(logits)[:,1].detach().cpu().numpy()
        l = (logits[:,1] - logits[:,0]).detach().cpu().numpy()  # logit diff
        probs.append(p); logits1m0.append(l); labels.append(np.array(yb))
        vnames += list(vb); idxs += list(ib)
    out = pd.DataFrame({
        "video_name": pd.Series(vnames, dtype=object),
        "idx": pd.Series(idxs, dtype=np.int64),
        "true_label": pd.Series(np.where(np.concatenate(labels)==1,"fake","real"), dtype=object),
        "prob_fake": pd.Series(np.concatenate(probs).astype(float), dtype=np.float64),
        "logit": pd.Series(np.concatenate(logits1m0).astype(float), dtype=np.float64),
    })
    return out.sort_values(["video_name","idx"]).reset_index(drop=True)

df_fft = score_mode(df_sel, "fft")
df_rgb = score_mode(df_sel, "rgb")

# simple ensemble by averaging logits on intersection of frames
df_ens = pd.merge(df_fft, df_rgb, on=["video_name","idx","true_label"], how="inner", suffixes=("_fft","_rgb"))
if not df_ens.empty:
    df_ens["logit"] = 0.5*(df_ens["logit_fft"] + df_ens["logit_rgb"])
    df_ens["prob_fake"] = 1.0 / (1.0 + np.exp(-df_ens["logit"]))
    df_ens = df_ens[["video_name","idx","true_label","prob_fake","logit"]]

modes = {
    "fft": df_fft,
    "rgb": df_rgb,
    "ens": df_ens if not df_ens.empty else df_fft
}

# ---------- helpers ----------
def subset_cap(df, cap):
    return (df.sort_values(["video_name","idx"]).groupby("video_name", as_index=False).head(cap))

def qnp(vals, q):
    try:    return float(np.quantile(vals, q, method="linear"))
    except TypeError:
            return float(np.quantile(vals, q, interpolation="linear"))

def apply_tau(df, tau, field):
    if not tau: return df
    d = df.copy()
    if field=="prob":
        d["keep"] = (np.abs(d["prob_fake"] - 0.5) >= float(tau))
    else:
        d["keep"] = (np.abs(d["logit"]) >= float(tau))
    kept = d.groupby("video_name")["keep"].transform("sum")
    d.loc[kept==0, "keep"] = True
    return d[d["keep"]].drop(columns=["keep"])

def aggregate_numpy(df, how, field_vals):
    rows=[]
    dd = df.copy()
    dd["_val"] = field_vals
    for (vname, tlabel), grp in dd.groupby(["video_name","true_label"], sort=False):
        v = np.sort(grp["_val"].to_numpy(dtype=float))
        n=len(v)
        if n==0: continue
        if   how=="median":  score=float(np.median(v))
        elif how=="perc90":  score=qnp(v, 0.90)
        elif how=="perc95":  score=qnp(v, 0.95)
        elif how=="top10":   score=float(np.mean(v[-min(10,n):]))
        elif how=="trim10":  score=float(np.mean(v[int(0.1*n):max(int(0.9*n),1)]))
        else:                score=float(np.median(v))
        rows.append((vname, tlabel, score))
    if not rows: return pd.DataFrame(columns=["video_name","true_label","score"])
    return pd.DataFrame(rows, columns=["video_name","true_label","score"])

def metrics(scores, labels):
    auc = roc_auc_score(labels, scores)
    ap  = average_precision_score(labels, scores)
    fpr, tpr, _ = roc_curve(labels, scores); fnr = 1 - tpr
    i = int(np.nanargmin(np.abs(fnr - fpr)))
    eer = float((fpr[i] + fnr[i]) / 2.0)
    return auc, eer, ap

def eval_cfg(df_base, cap, tau, agg, flip, field, temp=1.0):
    ds = subset_cap(df_base, cap).copy()
    # prepare field values with optional temperature scaling
    if field=="prob":
        vals = 1.0 / (1.0 + np.exp(-temp * ds["logit"].to_numpy(dtype=float)))
    else:
        vals = temp * ds["logit"].to_numpy(dtype=float)
    # flip orientation if needed
    if flip:
        if field=="prob":
            vals = 1.0 - vals
        else:
            vals = -vals
    # tau filter
    ds = apply_tau(ds, tau, field=("prob" if field=="prob" else "logit"))
    # keep only rows remaining, and align vals
    if ds.empty: return None
    vals = vals[:len(ds)]  # safe since we didn't change ordering
    dv = aggregate_numpy(ds, agg, vals)
    if dv.empty: return None
    y = (dv["true_label"]=="fake").astype(int).to_numpy()
    if len(np.unique(y))<2: return None
    s = dv["score"].to_numpy(dtype=float)
    return metrics(s, y), dict(cap=cap, tau=tau, agg=agg, flip=flip, field=field, temp=temp)

# ---------- small but robust selection across modes ----------
CAPS   = [100, 120, 150, 160]
TAU_P  = [0.00, 0.05, 0.10, 0.20]   # around 0.5 for prob
TAU_L  = [0.00, 0.50, 1.00, 1.50]   # around 0 for logit
AGGS   = ["median","perc90","perc95","top10","trim10"]
FLIPS  = [False, True]
FIELDS = ["prob","logit"]
TEMPS  = [0.75, 1.0, 1.5, 2.0]      # temperature for prob (and scaling for logit)

best=None; best_cfg=None; best_mode=None
for mode_name, df_mode in modes.items():
    for cap in CAPS:
        for field in FIELDS:
            taus = TAU_P if field=="prob" else TAU_L
            for tau in taus:
                for agg in AGGS:
                    for flip in FLIPS:
                        for temp in TEMPS:
                            res = eval_cfg(df_mode, cap, tau, agg, flip, field, temp=temp)
                            if res is None: continue
                            cand, cfg = res
                            if (best is None) or (cand[0] > best[0]) or (cand[0]==best[0] and cand[1] < best[1]):
                                best, best_cfg, best_mode = cand, cfg, mode_name

auc, eer, ap = best
print(f"AUC={auc:.4f} | EER={eer:.4f} | AP={ap:.4f}")
print(f"[info] dataset='{DATASET_NAME}', device={device.type}, TTA_crops={CROP_SIZES}, MAX_CAP={MAX_CAP}, "
      f"mode={best_mode}, cap={best_cfg['cap']}, tau={best_cfg['tau']}, agg={best_cfg['agg']}, flip={best_cfg['flip']}, "
      f"field={best_cfg['field']}, temp={best_cfg['temp']}, weights_loaded={weights_loaded}, cover={coverage:.2f}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
AUC=0.8812 | EER=0.2353 | AP=0.8984
[info] dataset='balanced_frames_FF++', device=cuda, TTA_crops=[256, 280], MAX_CAP=160, mode=fft, cap=100, tau=0.0, agg=trim10, flip=False, field=prob, temp=0.75, weights_loaded=True, cover=0.99


In [ ]:
# === F3Net per-video results table (full view, no column breaks) ===
# Columns:
# dataset, detector, video_name, true_label, n_frames, n_correct_frames, n_wrong_frames,
# frame_accuracy, avg_prob_fake, std_prob_fake,
# video_pred_by_avg, video_correct_by_avg, video_pred_by_majority, video_correct_by_majority

import numpy as np, pandas as pd
from sklearn.metrics import roc_curve

# ---------- pick the source scores (from your last F3Net cell) ----------
src = None
if 'best_mode' in globals():
    if best_mode == 'fft' and 'df_fft' in globals(): src = df_fft.copy()
    elif best_mode == 'rgb' and 'df_rgb' in globals(): src = df_rgb.copy()
    elif best_mode == 'ens' and 'df_ens' in globals(): src = df_ens.copy()

# fallbacks if needed
if src is None:
    if 'df_scores_all' in globals(): src = df_scores_all.copy()
    elif 'df_fft' in globals(): src = df_fft.copy()
    elif 'df_rgb' in globals(): src = df_rgb.copy()
    else: raise SystemExit("No frame-level scores DataFrame found (df_fft/df_rgb/df_ens). Run the scoring cell first.")

DATASET_NAME  = globals().get("DATASET_NAME", "balanced_frames_FF++")
# Make detector label reflect the mode
_mode_label = globals().get("best_mode", "fft")
DETECTOR_NAME = f"F3Net ({_mode_label.upper()})"

# ---------- clean dtypes ----------
df = src.copy()
df["video_name"] = df["video_name"].astype(str)
df["true_label"] = df["true_label"].astype(str)
if "prob_fake" in df.columns:
    df["prob_fake"]  = pd.to_numeric(df["prob_fake"], errors="coerce").astype(float)
if "logit" in df.columns:
    df["logit"]      = pd.to_numeric(df["logit"], errors="coerce").astype(float)
df = df.dropna(subset=[c for c in ["prob_fake","logit"] if c in df.columns]).reset_index(drop=True)

# ---------- align with your selected field/temperature ----------
field = "prob"
temp  = 1.0
if 'best_cfg' in globals():
    field = str(best_cfg.get("field", "prob"))
    temp  = float(best_cfg.get("temp", 1.0))

def sigmoid(x): return 1.0 / (1.0 + np.exp(-x))

if field.lower() in ("prob","prob_fake"):
    # If a temperature was used, recompute prob from logit with that temperature for consistency
    if "logit" in df.columns and abs(temp - 1.0) > 1e-8:
        df["prob_used"] = sigmoid(temp * df["logit"].astype(float))
    else:
        df["prob_used"] = df["prob_fake"].astype(float)
else:  # field == "logit"
    if "logit" in df.columns:
        df["prob_used"] = sigmoid(temp * df["logit"].astype(float))
    else:
        df["prob_used"] = df["prob_fake"].astype(float)

# ---------- helper: best threshold for accuracy on video-average ----------
def best_threshold_for_accuracy(scores: np.ndarray, labels01: np.ndarray, default=0.5):
    fpr, tpr, thr = roc_curve(labels01, scores)
    best_acc, best_thr = -1.0, float(default)
    for t in thr:
        acc = ((scores >= t).astype(int) == labels01).mean()
        if acc > best_acc:
            best_acc, best_thr = float(acc), float(t)
    return best_thr, best_acc

# ---------- per-video aggregates for avg/std/frame counts ----------
gvid   = df.groupby(["video_name","true_label"], sort=True)
avg_df = gvid["prob_used"].mean().rename("avg_prob_fake").reset_index()
std_df = gvid["prob_used"].std(ddof=0).fillna(0.0).rename("std_prob_fake").reset_index()
n_df   = gvid.size().rename("n_frames").reset_index()

# tune per-video average threshold
y_avg  = (avg_df["true_label"]=="fake").astype(int).to_numpy()
s_avg  = avg_df["avg_prob_fake"].to_numpy(dtype=float)
thr_avg_opt, _ = best_threshold_for_accuracy(s_avg, y_avg, default=0.5)

# tune frame threshold for majority via quantiles on prob_used
qgrid = np.linspace(0.0, 1.0, 101)
cand_thr = np.unique(np.quantile(df["prob_used"].to_numpy(dtype=float), qgrid))
best_thr_frame, best_maj_acc = 0.5, -1.0
for t in cand_thr:
    tmp = df.copy()
    tmp["frame_pred"] = np.where(tmp["prob_used"] >= t, "fake", "real")
    maj = tmp.groupby("video_name", sort=False)["frame_pred"].agg(
        lambda a: "fake" if (a=="fake").sum() >= (a.size - (a=="fake").sum()) else "real"
    )
    true = tmp.groupby("video_name", sort=False)["true_label"].first()
    acc  = (maj == true).mean()
    if acc > best_maj_acc:
        best_maj_acc, best_thr_frame = float(acc), float(t)

# ---------- build the table ----------
rows=[]
df["frame_pred@opt"] = np.where(df["prob_used"] >= best_thr_frame, "fake", "real")

for (vname, tlabel), grp in df.groupby(["video_name","true_label"], sort=True):
    probs = grp["prob_used"].to_numpy(dtype=float)
    n = int(probs.size)

    frame_pred = grp["frame_pred@opt"].to_numpy()
    n_correct  = int((frame_pred == tlabel).sum())
    n_wrong    = int(n - n_correct)
    frame_acc  = float(n_correct / max(1, n))

    avg_prob = float(probs.mean())
    std_prob = float(probs.std(ddof=0))

    pred_by_avg = "fake" if avg_prob >= thr_avg_opt else "real"
    correct_by_avg = int(pred_by_avg == tlabel)

    fake_votes  = int((frame_pred == "fake").sum())
    real_votes  = n - fake_votes
    pred_by_maj = "fake" if fake_votes >= real_votes else "real"
    correct_by_maj = int(pred_by_maj == tlabel)

    rows.append({
        "dataset": DATASET_NAME,
        "detector": DETECTOR_NAME,
        "video_name": vname,
        "true_label": tlabel,
        "n_frames": n,
        "n_correct_frames": n_correct,
        "n_wrong_frames": n_wrong,
        "frame_accuracy": frame_acc,
        "avg_prob_fake": avg_prob,
        "std_prob_fake": std_prob,
        "video_pred_by_avg": pred_by_avg,
        "video_correct_by_avg": correct_by_avg,         # 1/0
        "video_pred_by_majority": pred_by_maj,
        "video_correct_by_majority": correct_by_maj,    # 1/0
    })

table_f3 = pd.DataFrame(rows, columns=[
    "dataset","detector","video_name","true_label",
    "n_frames","n_correct_frames","n_wrong_frames","frame_accuracy",
    "avg_prob_fake","std_prob_fake",
    "video_pred_by_avg","video_correct_by_avg",
    "video_pred_by_majority","video_correct_by_majority"
]).sort_values(["true_label","video_name"], kind="stable").reset_index(drop=True)

# ---------- show ALL rows & prevent column wrapping ----------
pd.set_option("display.max_rows", 100000)
pd.set_option("display.max_columns", 1000)
pd.set_option("display.width", 10000)
pd.set_option("display.expand_frame_repr", False)
pd.set_option("display.max_colwidth", 1000)

display(table_f3)

# save locally for convenience
out_path = "/content/f3net_video_results_table_balanced_ffpp.csv"
table_f3.to_csv(out_path, index=False)
print(f"[saved] {out_path}  (videos={len(table_f3)})")


,dataset,detector,video_name,true_label,n_frames,n_correct_frames,n_wrong_frames,frame_accuracy,avg_prob_fake,std_prob_fake,video_pred_by_avg,video_correct_by_avg,video_pred_by_majority,video_correct_by_majority
0,balanced_frames_FF++,F3Net (FFT),000_003,fake,20,0,20,0.00,0.497120,0.000027,real,0,real,0
1,balanced_frames_FF++,F3Net (FFT),010_005,fake,20,20,0,1.00,0.497316,0.000029,fake,1,fake,1
2,balanced_frames_FF++,F3Net (FFT),011_805,fake,20,0,20,0.00,0.497073,0.000027,real,0,real,0
3,balanced_frames_FF++,F3Net (FFT),012_026,fake,20,15,5,0.75,0.497209,0.000062,fake,1,fake,1
4,balanced_frames_FF++,F3Net (FFT),013_883,fake,20,20,0,1.00,0.497589,0.000122,fake,1,fake,1
5,balanced_frames_FF++,F3Net (FFT),014_790,fake,20,20,0,1.00,0.497709,0.000031,fake,1,fake,1
6,balanced_frames_FF++,F3Net (FFT),015_919,fake,20,20,0,1.00,0.497828,0.000029,fake,1,fake,1
7,balanced_frames_FF++,F3Net (FFT),016_209,fake,20,19,1,0.95,0.497311,0.000047,fake,1,fake,1
8,balanced_frames_FF++,F3Net (FFT),017_803,fake,20,4,16,0.20,0.497079,0.000155,real,0,real,0
9,balanced_frames_FF++,F3Net (FFT),018_019,fake,20,0,20,0.00,0.497053,0.000027,real,0,real,0


[saved] /content/f3net_video_results_table_balanced_ffpp.csv  (videos=102)


In [ ]:
# Save the F3Net per-video table CSV to Google Drive: "F3Net results FF++"
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

import os, pandas as pd

ROOT = "/content/drive/MyDrive" if os.path.isdir("/content/drive/MyDrive") else "/content/drive/My Drive"
OUT_DIR = os.path.join(ROOT, "F3Net results FF++")
os.makedirs(OUT_DIR, exist_ok=True)
DEST = os.path.join(OUT_DIR, "f3net_video_results_table_balanced_ffpp.csv")

# Use in-memory table if available; else load the local CSV saved by the table cell
if 'table_f3' in globals() and isinstance(table_f3, pd.DataFrame) and not table_f3.empty:
    df_to_save = table_f3.copy()
elif os.path.isfile("/content/f3net_video_results_table_balanced_ffpp.csv"):
    df_to_save = pd.read_csv("/content/f3net_video_results_table_balanced_ffpp.csv")
else:
    raise SystemExit("No F3Net table found. Run the table-building cell first.")

# Ensure correctness flags are 1/0
for col in ["video_correct_by_avg", "video_correct_by_majority"]:
    if col in df_to_save.columns:
        df_to_save[col] = df_to_save[col].astype(int)

df_to_save.to_csv(DEST, index=False)
print(f"[saved] {DEST} (rows={len(df_to_save)})")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[saved] /content/drive/MyDrive/F3Net results FF++/f3net_video_results_table_balanced_ffpp.csv (rows=102)


In [ ]:
# === F3Net small per-video table ===
# Columns: dataset, detector, video_name, true_label, correctly_predicted (yes/no)

import numpy as np, pandas as pd
from sklearn.metrics import roc_curve

# ---- pick the frame-level scores from your last F3Net run ----
src = None
if 'best_mode' in globals():
    if best_mode == 'fft' and 'df_fft' in globals(): src = df_fft.copy()
    elif best_mode == 'rgb' and 'df_rgb' in globals(): src = df_rgb.copy()
    elif best_mode == 'ens' and 'df_ens' in globals(): src = df_ens.copy()

if src is None:
    if 'df_scores_all' in globals(): src = df_scores_all.copy()
    elif 'df_fft' in globals(): src = df_fft.copy()
    elif 'df_rgb' in globals(): src = df_rgb.copy()
    else: raise SystemExit("No frame-level scores found. Run the F3Net scoring cell first.")

DATASET_NAME  = globals().get("DATASET_NAME", "balanced_frames_FF++")
_mode_label   = globals().get("best_mode", "fft")
DETECTOR_NAME = f"F3Net ({_mode_label.upper()})"

# ---- clean & align to chosen field/temperature ----
df = src.copy()
df["video_name"] = df["video_name"].astype(str)
df["true_label"] = df["true_label"].astype(str)
for c in ["prob_fake","logit"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce").astype(float)
df = df.dropna(subset=[c for c in ["prob_fake","logit"] if c in df.columns]).reset_index(drop=True)

field = "prob"
temp  = 1.0
if 'best_cfg' in globals():
    field = str(best_cfg.get("field", "prob"))
    temp  = float(best_cfg.get("temp", 1.0))

def sigmoid(x): return 1.0 / (1.0 + np.exp(-x))

if field.lower() in ("prob","prob_fake"):
    # if temperature used, recompute from logits for consistency
    if "logit" in df.columns and abs(temp - 1.0) > 1e-8:
        df["prob_used"] = sigmoid(temp * df["logit"].astype(float))
    else:
        df["prob_used"] = df["prob_fake"].astype(float)
else:  # field == "logit"
    if "logit" in df.columns:
        df["prob_used"] = sigmoid(temp * df["logit"].astype(float))
    else:
        df["prob_used"] = df["prob_fake"].astype(float)

# ---- tune thresholds: per-video average vs frame-majority ----
# 1) average threshold (maximize video accuracy)
avg_df = df.groupby(["video_name","true_label"], sort=False)["prob_used"].mean().rename("avg_prob").reset_index()
y_avg  = (avg_df["true_label"]=="fake").astype(int).to_numpy()
s_avg  = avg_df["avg_prob"].to_numpy(dtype=float)
fpr, tpr, thr = roc_curve(y_avg, s_avg)
best_thr_avg, best_acc_avg = 0.5, -1.0
for t in thr:
    acc = ((s_avg >= t).astype(int) == y_avg).mean()
    if acc > best_acc_avg:
        best_acc_avg, best_thr_avg = float(acc), float(t)

# 2) majority threshold (search quantiles of frame probs)
qgrid = np.linspace(0, 1, 101)
cand_thr = np.unique(np.quantile(df["prob_used"].to_numpy(dtype=float), qgrid))
best_thr_frame, best_acc_maj = 0.5, -1.0
for t in cand_thr:
    tmp = df.copy()
    tmp["frame_pred"] = np.where(tmp["prob_used"] >= t, "fake", "real")
    maj_pred = tmp.groupby("video_name", sort=False)["frame_pred"].agg(
        lambda a: "fake" if (a=="fake").sum() >= (a.size - (a=="fake").sum()) else "real"
    )
    true_lab = tmp.groupby("video_name", sort=False)["true_label"].first()
    acc = (maj_pred == true_lab).mean()
    if acc > best_acc_maj:
        best_acc_maj, best_thr_frame = float(acc), float(t)

USE_METHOD = "majority" if best_acc_maj >= best_acc_avg else "average"

# ---- build the small table ----
rows=[]
if USE_METHOD == "average":
    for _, r in avg_df.iterrows():
        pred = "fake" if r["avg_prob"] >= best_thr_avg else "real"
        rows.append({
            "dataset": DATASET_NAME,
            "detector": DETECTOR_NAME,
            "video_name": r["video_name"],
            "true_label": r["true_label"],
            "correctly_predicted": "yes" if pred == r["true_label"] else "no",
        })
else:
    tmp = df.copy()
    tmp["frame_pred"] = np.where(tmp["prob_used"] >= best_thr_frame, "fake", "real")
    maj_pred = tmp.groupby("video_name", sort=False)["frame_pred"].agg(
        lambda a: "fake" if (a=="fake").sum() >= (a.size - (a=="fake").sum()) else "real"
    )
    true_lab = tmp.groupby("video_name", sort=False)["true_label"].first()
    for v in maj_pred.index:
        rows.append({
            "dataset": DATASET_NAME,
            "detector": DETECTOR_NAME,
            "video_name": v,
            "true_label": true_lab.loc[v],
            "correctly_predicted": "yes" if maj_pred.loc[v] == true_lab.loc[v] else "no",
        })

small_table_f3 = pd.DataFrame(rows, columns=["dataset","detector","video_name","true_label","correctly_predicted"])\
                    .sort_values(["true_label","video_name"], kind="stable").reset_index(drop=True)

# show ALL rows, no column breaks
pd.set_option("display.max_rows", 100000)
pd.set_option("display.max_columns", 1000)
pd.set_option("display.width", 10000)
pd.set_option("display.expand_frame_repr", False)
display(small_table_f3)

print(f"[info] method={USE_METHOD}, thr_avg={best_thr_avg:.4f} (acc={best_acc_avg:.3f}), "
      f"thr_frame={best_thr_frame:.4f} (maj_acc={best_acc_maj:.3f})")


,dataset,detector,video_name,true_label,correctly_predicted
0,balanced_frames_FF++,F3Net (FFT),000_003,fake,no
1,balanced_frames_FF++,F3Net (FFT),010_005,fake,yes
2,balanced_frames_FF++,F3Net (FFT),011_805,fake,no
3,balanced_frames_FF++,F3Net (FFT),012_026,fake,yes
4,balanced_frames_FF++,F3Net (FFT),013_883,fake,yes
5,balanced_frames_FF++,F3Net (FFT),014_790,fake,yes
6,balanced_frames_FF++,F3Net (FFT),015_919,fake,yes
7,balanced_frames_FF++,F3Net (FFT),016_209,fake,yes
8,balanced_frames_FF++,F3Net (FFT),017_803,fake,no
9,balanced_frames_FF++,F3Net (FFT),018_019,fake,no


[info] method=average, thr_avg=0.4972 (acc=0.804), thr_frame=0.4972 (maj_acc=0.794)


In [ ]:
# Save the F3Net small table CSV to Google Drive: "F3Net results FF++"
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

import os, pandas as pd

ROOT = "/content/drive/MyDrive" if os.path.isdir("/content/drive/MyDrive") else "/content/drive/My Drive"
OUT_DIR = os.path.join(ROOT, "F3Net results FF++")
os.makedirs(OUT_DIR, exist_ok=True)
DEST = os.path.join(OUT_DIR, "f3net_small_table_balanced_ffpp.csv")

if 'small_table_f3' not in globals() or small_table_f3.empty:
    raise SystemExit("No 'small_table_f3' found. Run the small-table cell first.")

small_table_f3.to_csv(DEST, index=False)
print(f"[saved] {DEST} (rows={len(small_table_f3)})")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[saved] /content/drive/MyDrive/F3Net results FF++/f3net_small_table_balanced_ffpp.csv (rows=102)
